In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
# sys.path = ['/home/jxm3/research/retrieval/inversion'] + sys.path
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pandas as pd

emb_df = pd.read_csv("/home/jxm3/research/retrieval/2023_05_15_luar_embedding_list.csv")
emb_df.head(1)

,Unnamed: 0,query_index,embedding
0,0,2,"[0.15197467803955078, -0.28477445244789124, 0...."


In [3]:
import numpy as np

embeddings = emb_df["embedding"].tolist()
embeddings = list(map(eval, embeddings))
embeddings = np.array(embeddings)

In [7]:
# luar model with msl 128:
# https://wandb.ai/jack-morris/emb-inv-1/runs/a812ef9463f48360967d375c924c461e/overview?workspace=user-jxmorris12


import analyze_utils

checkpoint_folder = '/home/jxm3/research/retrieval/inversion/saves/947b54370a00d767cb28b2e71a2645b0/a812ef9463f48360967d375c924c461e'
args_str = '--per_device_train_batch_size 128 --per_device_eval_batch_size 128 --max_seq_length 128 --model_name_or_path t5-base --embedder_model_name paraphrase-distilroberta --num_repeat_tokens 16 --embedder_no_grad True --exp_group_name mar15-luar --learning_rate 0.0002 --freeze_strategy none --embedder_fake_with_zeros False --num_train_epochs 24 --max_eval_samples 1000 --eval_steps 4000 --warmup_steps 100000 --bf16=1 --use_wandb=1 --dataset_name luar_reddit --use_frozen_embeddings_as_input True --exp_name luar128'
trainer = analyze_utils.load_trainer(checkpoint_folder, args_str, sanity_decode=False)

trainer.evaluate()

Set train_args.dataloader_num_workers = 4
Loading trainer for analysis – setting --do_eval=1


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

Running tokenizer on dataset:   0%|          | 0/1089648 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/169936 [00:00<?, ? examples/s]

...

Running tokenizer on dataset:   0%|          | 0/11332 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/11332 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/72831 [00:00<?, ? examples/s]

[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'eval_perplexity': -1,
 'eval_runtime': 0.0075,
 'eval_samples_per_second': 667.755,
 'eval_steps_per_second': 133.551}

In [8]:
trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
trainer.evaluate()

{'eval_perplexity': -1,
 'eval_runtime': 0.007,
 'eval_samples_per_second': 712.832,
 'eval_steps_per_second': 142.566}

In [9]:
import torch

emb_dataloader = torch.utils.data.DataLoader(
    embeddings,
    batch_size=trainer.args.eval_batch_size,
    num_workers=trainer.args.dataloader_num_workers,
    pin_memory=trainer.args.dataloader_pin_memory,
)

In [10]:
import copy
import tqdm

output_text = []

gen_kwargs = copy.copy(trainer.gen_kwargs)
for emb_batch in tqdm.tqdm(emb_dataloader):
    inputs = { "frozen_embeddings": emb_batch.float(), "embedder_input_ids": torch.ones(len(emb_batch)), "embedder_attention_mask": torch.ones(len(emb_batch)) }
    inputs_cuda = {k: v.to(trainer.args.device) for k, v in inputs.items()}
    gen_kwargs["max_length"] = 128
    generated_ids = trainer.generate(
        inputs=inputs_cuda, generation_kwargs=gen_kwargs
    )
    output_text.extend(
        trainer.embedder_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    )   

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.53s/it]


In [11]:
text_and_embeddings = pd.DataFrame(
    zip(output_text, embeddings), columns=['text', 'embedding']
)
text_and_embeddings.to_parquet('2023_05_15_luar_reconstruction.parquet.gzip', compression='gzip')

In [13]:
text_and_embeddings.tail()

,text,embedding
21,� does if it Hak his to of product women by hi...,"[0.19542090594768524, 0.030160320922732353, 0...."
22,during toode the� place asen) of manager's of...,"[-0.29221123456954956, -0.1277933418750763, 0...."
23,"� place it from to operating,'m has you people...","[-0.12761323153972626, -0.047798529267311096, ..."
24,9 from to if and making's protections fight it...,"[0.34192952513694763, 0.04247118532657623, 0.0..."
25,during bye over and making's of Glasgow of ge...,"[-0.03345293179154396, -0.19010047614574432, -..."
